## Redes Neurais Artificiais 2020.1

**Disciplina**: Redes Neurais Artificiais 2020.1  
**Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
**Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta terceira parte do Projeto Prático 3 diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Testando Redes Neurais sem os Atributos Categórios

1. Abra a base de dados em questão
2. Elimine todas as colunas relativas aos atributos categóricos
3. Armazene o atributo alvo em uma variável y e os atributos preditores em uma variável X
4. Efetue uma partição holdout 70/30 com o sklearn, distribuindo os exemplos de maneira aleatória
5. Efetue o escalonamento dos atributos

### Escalonando os atributos

O treinamento de uma rede neural artificial é mais eficiente quando os valores que lhes são fornecidos como entrada são pequenos, pois isto favorece a convergência. Isto é feito escalonando-se todos os atributos para o intervalo [0,1], mas precisa ser feito de maneira cautelosa, para que informações do conjunto de teste não sejam fornecidas no treinamento.

Há duas estratégias para tal escalonamento: normalização e padronização. Ambas possuem características particulares, vantagens e limitações, como é possível ver aqui: https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/


No nosso caso, vamos usar a padronização. Assim, com os atributos preditores do treinamento, isto é, X_train, deve-se subtrair a média e dividir pelo desvio padrão:

X_train_std = (X_train - np.mean(X_train))/np.std(X_train)

Em seguida, o mesmo deve ser feito com os atributos preditores do conjunto de testes, mas com padronização relativa ao conjunto de treinamento:

X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

Se todo o conjunto X for utilizado na padronização, a rede neural receberá informações do conjunto de teste por meio da média e variância utilizada para preparar os dados de treinamento, o que não é desejável.


### Continuando

5. Treine uma rede neural multilayer perceptron para este problema com uma única camada e dez neurônios  
    5.1 Utilize a função de ativação ReLU  
    5.2 Utilize o solver Adam    
    5.3 Imprima o passo a passo do treinamento    
    5.4 Utilize o número máximo de épocas igual a 300  
6. Com o modelo em questão, após o treinamento, apresente:  
    6.1 Matriz de confusão para o conjunto de teste  
    6.2 Acurácia  
    6.3 F-Score  
    6.4 Precisão  
    6.5 Revocação  
7. Repita o treinamento da mesma rede anterior sem imprimir o passo a passo (verbose False) por 100 vezes  
    7.1 Cada uma destas repetições deve ser feita com uma nova partição Holdout  
    7.2 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  
8. Repita por 100 vezes o treinamento desta mesma rede, mas utilizando o otimizador SGD  
    8.1 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  
9. Houve influência da escolha do otimizador no desempenho da rede?

In [1]:
## Reservado para a importação de bibliotecas

from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.offline as py
import plotly.graph_objects as go
import math
import random
from prettytable import PrettyTable  
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [2]:
# Montagem do drive para o carregamento da base de dados por meio do google colab
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
# Leitura do dataset covtype.csv

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/covtype.csv', sep=',')  # caso use google colab
# df = pd.read_csv('./covtype.csv')                                               # caso faça localmente pelo jupyter

#### Preparação dos dados

In [4]:
# Eliminacao das colunas relativas aos atributos categoricos

atributosCategoricos = []
for i in range(40):                                                               # loop para preencher um vetor com os atributos categoricos
  if i <=3:
    atributosCategoricos.append("Wilderness_Area"+str(i+1))
  atributosCategoricos.append("Soil_Type"+str(i+1))

df = df.drop(columns=atributosCategoricos)                                        # delecao dos atributos categoricos

In [5]:
y_alvo = df["Cover_Type"]                                                         # separacao do atributo alvo
x_preditor = df.drop(columns=["Cover_Type"])                                      # separacao dos atributos preditores

# Particao holdout para teste e treino
x_train, x_test, y_train, y_test = train_test_split(                              # Criacao das particoes para treino e teste com o auxilio da biblioteca sklearn
    x_preditor, y_alvo, test_size=0.3, train_size=0.7)

In [6]:
# Escalonamento usando o metodo da padronização

X_train_std = (x_train - np.mean(x_train))/np.std(x_train)                        # Escalonamento do conjunto de treino 
X_test_std = (x_test - np.mean(x_train))/np.std(x_train)                          # escalonamento do conjunto de teste levando em consideracao o conjunto de treino

#### Criação e treinamento da rede neural

In [16]:
# criacao e treino da rede neural multilayer perceptron
clf = MLPClassifier(hidden_layer_sizes=(10),activation="relu", solver="adam", 
                    random_state=1, max_iter=300, 
                    verbose=True).fit(X_train_std, y_train)

Iteration 1, loss = 0.92591820
Iteration 2, loss = 0.70723923
Iteration 3, loss = 0.68876529
Iteration 4, loss = 0.68025876
Iteration 5, loss = 0.67412817
Iteration 6, loss = 0.66988273
Iteration 7, loss = 0.66729017
Iteration 8, loss = 0.66543756
Iteration 9, loss = 0.66389991
Iteration 10, loss = 0.66283764
Iteration 11, loss = 0.66187217
Iteration 12, loss = 0.66094189
Iteration 13, loss = 0.66010021
Iteration 14, loss = 0.65948238
Iteration 15, loss = 0.65880299
Iteration 16, loss = 0.65824638
Iteration 17, loss = 0.65783200
Iteration 18, loss = 0.65745867
Iteration 19, loss = 0.65706568
Iteration 20, loss = 0.65674884
Iteration 21, loss = 0.65654777
Iteration 22, loss = 0.65634320
Iteration 23, loss = 0.65606280
Iteration 24, loss = 0.65580755
Iteration 25, loss = 0.65566029
Iteration 26, loss = 0.65553128
Iteration 27, loss = 0.65535162
Iteration 28, loss = 0.65533766
Iteration 29, loss = 0.65520549
Iteration 30, loss = 0.65506871
Iteration 31, loss = 0.65494528
Iteration 32, los

In [ ]:
# obtencao das respostas para o conjunto de teste
y_pred = clf.predict(X_test_std)

# print da matriz de confusao
matrizConfusao = confusion_matrix(y_test, y_pred)                                 # calculo da matriz de confusao

table = PrettyTable(["","1", "2", "3", "4", "5", "6", "7"])                       # Cria a tabela com as colunas de 1 a 7
table.padding_width = 1

for i in range(len(matrizConfusao)):                                              # loop para preencher a tabela com os dados
  table.add_row(np.concatenate([[str(i+1)], matrizConfusao[i]]))

print("----------------- MATRIZ DE CONFUSAO -----------------")                   # print da matriz de confusao
print(table)

print("F1-SCORE:  {}".format(round(f1_score(y_test, y_pred,                        # calculo do f-score
                                           average='macro'), 4))) 
print("ACURACIA:  {}".format(round(accuracy_score(y_test, y_pred), 4)))            # calculo da acuracia
print("PRECISION: {}".format(round(precision_score(y_test, y_pred,                # calculo da precisao
                                                   average='macro'), 4)))
print("RECALL:    {}".format(round(recall_score(y_test, y_pred,                      # calculo da revocacao
                                             average='macro'), 4)))

----------------- MATRIZ DE CONFUSAO -----------------
+---+-------+-------+------+-----+-----+------+------+
|   |   1   |   2   |  3   |  4  |  5  |  6   |  7   |
+---+-------+-------+------+-----+-----+------+------+
| 1 | 43913 | 18200 |  8   |  0  |  8  |  9   | 1418 |
| 2 | 15281 | 67963 | 1156 |  0  | 100 | 475  | 103  |
| 3 |   0   |  1858 | 7512 | 131 |  0  | 1137 |  0   |
| 4 |   0   |   4   | 375  | 271 |  0  | 145  |  0   |
| 5 |   15  |  2547 |  46  |  0  | 323 |  10  |  0   |
| 6 |   0   |  1463 | 2299 |  54 |  0  | 1411 |  0   |
| 7 |  2843 |   28  |  0   |  0  |  0  |  0   | 3198 |
+---+-------+-------+------+-----+-----+------+------+
F1-SCORE:  0.5288
ACURACIA:  0.7148
PRECISION: 0.6527
RECALL:    0.4919


In [ ]:
f1scores = np.zeros(100)
acuracias = np.zeros(100)

for i in range (100):
  x_train, x_test, y_train, y_test = train_test_split(                              # Criacao das particoes para treino e teste com o auxilio da biblioteca sklearn
    x_preditor, y_alvo, test_size=0.3, train_size=0.7)
  
  X_train_std = (x_train - np.mean(x_train))/np.std(x_train)                        # Escalonamento do conjunto de treino 
  X_test_std = (x_test - np.mean(x_train))/np.std(x_train)  

  clf = MLPClassifier(hidden_layer_sizes=(10),activation="relu", solver="adam",     # criacao da rede neural multilayer perceptron
                      max_iter=300, 
                      verbose=False).fit(X_train_std, y_train)

  clf.fit(X_train_std, y_train)                                                     # treino da rede neural 

  y_pred = clf.predict(X_test_std)                                                  # predicao para o conjuto de teste escalonado

  f1scores[i] = f1_score(y_test, y_pred, average='macro')                           # armazenamento do valor de f-score
  acuracias[i] = accuracy_score(y_test, y_pred)                                     # armazenamento do valor de acuracia

# calculo das medias e desvios padrao dos dados de acuracia e f-score
print("Média de acurácia:      {}".format(round(acuracias.mean(), 5)))
print("Desvio Padrão acurácia: {}".format(round(acuracias.std(), 5)))
print("Média de F-Score:       {}".format(round(f1scores.mean(), 5)))
print("Desvio Padrão F-Score:  {}".format(round(f1scores.std(), 5)))

Média de acurácia:      0.71554
Desvio Padrão acurácia: 0.0021
Média de F-Score:       0.51661
Desvio Padrão F-Score:  0.01065


#### Treinamento da rede neural anterior levando em consideracao o **SOLVER** sendo `SGD`

In [ ]:
f1scores = np.zeros(100)
acuracias = np.zeros(100)

for i in range (100):
  x_train, x_test, y_train, y_test = train_test_split(                              # Criacao das particoes para treino e teste com o auxilio da biblioteca sklearn
    x_preditor, y_alvo, test_size=0.3, train_size=0.7)
  
  X_train_std = (x_train - np.mean(x_train))/np.std(x_train)                        # Escalonamento do conjunto de treino 
  X_test_std = (x_test - np.mean(x_train))/np.std(x_train)  

  clf = MLPClassifier(hidden_layer_sizes=(10),activation="relu", solver="sgd", 
                      max_iter=300, 
                      verbose=False).fit(X_train_std, y_train)

  y_pred = clf.predict(X_test_std)

  f1scores[i] = f1_score(y_test, y_pred, average='macro')
  acuracias[i] = accuracy_score(y_test, y_pred)

print("media acuracia:          {}".format(round(acuracias.mean(), 5)))
print("desvio padrao acuracia:  {}".format(round(acuracias.std(), 5)))
print("media f-score:           {}".format(round(f1scores.mean(), 5)))
print("desvio padrao f-score:   {}".format(round(f1scores.std(), 5)))

media acuracia:          0.7147
desvio padrao acuracia:  0.00301
media f-score:           0.48194
desvio padrao f-score:   0.01792


### Houve influência da escolha do otimizador no desempenho da rede?

Sim, como foi visto a partir dos dados de media e desvio padrão sobre a acuracia e o F-Score com os hiperparametros solver `adam e sgd`. O modelo que teve o solver sendo adam se saiu levemente melhor, obtendo uma maior pontuação para media de f-score e ligeiramente melhor para media de acuracia tambem.

## Discussão

Nos passos anteriores, você avaliou o desempenho de uma única rede neural que contém os seguintes parâmetros: uma única camada oculta com 10 neurônios e função de ativação ReLU. O otimizador utilizado, quer seja SGD ou ADAM, trata-se do algoritmo para aproximar o gradiente do erro. Neste sentido, a escolha do otimizador é um hiperparâmetro, pois diz respeito a como a rede neural definida previamente atuará "em tempo de execução"  durante o processo de treinamento. Também são hiperparâmetros a quantidade de épocas, a taxa de aprendizado inicial, dentre outros.

Cabe alientar também que você efetuou o treinamento desta rede por 100 vezes e apresentou os resultados em termos de média +- desvio padrão. Lembre-se que em uma rede neural há a inicialização aleatória de pesos e, em consequência, o desempenho delas está sujeito à uma flutuação estocástica. A execução destas múltiplas vezes faz com que eliminemos algum viés introduzido por uma boa ou má "sorte" na escolha de pesos no caso de uma única execução.

## Propondo Novas Arquiteturas

Variando  os parâmetros (uma ou duas camadas ocultas, com diferente números de neurônios em cada uma delas e a função de ativação) e o hiperparâmetros solver (Adam ou SGD) e o número de épocas (100,150 e 200), atenda ao que se pede:

1. Proponha 10 arquiteturas distintas de RNAs para o problema em questão, à sua escolha
2. Avalie cada uma das arquiteturas perante todos os hiperparâmetros apresentados por 100 vezes
3. Como resultado da avaliação, apresente:  
    3.1 Top-3 melhores redes no tocante à F-Score e Acurácia  
    3.2 Repetição em que houve o melhor desempenho de cada uma dessas redes: ilustre tp, tf, fp e fn  

#### Criação das configuracoes para as redes neurais

In [18]:
funcaoAtivação = ["identity", "logistic", "tanh", "relu"]                         # declaracao do vetor com os tipos de funcao de ativacao
hiperParametro = ["adam", "sgd"]                                                  # declaracao do vetor com o tipo do hiperparametro solver
epocas = [100, 150, 200]                                                          # declaracao do vetor com a quanitdade de epocas
camadas = [1, 2]
configuracoes = []                                                                # declaracao do vetor que armazenara as configuracoes geradas

for i in range (10):                                                              # loop para gerar aleatoriamente as configuracoes (dentro do especificado)
  tipoEpoca = random.randrange(1, 1000000, 1)%3
  tipoHiperParametro = random.randrange(1, 1000000, 1)%2
  tipoFuncaoAtivacao = random.randrange(1, 1000000, 1)%4
  xqtdCamadas = random.randrange(1, 1000000, 1)%2
  qtdNeuronios = random.randrange(1, 20, 1)
  
  aux = random.randrange(1, qtdNeuronios, 1)                                      # variavel auxiliar
  disposicaoNeuronios = qtdNeuronios if camadas[xqtdCamadas]==1 else (aux, qtdNeuronios-aux) # disposicao dos neuronios nas 2 camadas ocultas

  configuracoes.append({                                                          # armazenamento das configuracoes geradas
    "camadasOcultas": camadas[xqtdCamadas],
    "funcaoAtivacao": funcaoAtivação[tipoFuncaoAtivacao],
    "hiperparametro": hiperParametro[tipoHiperParametro],
    "epocas": epocas[tipoEpoca],
    "neuronios": qtdNeuronios,
    "disposicaoNeuronios": disposicaoNeuronios
  })

#### Apresentação das configuracoes de redes geradas

In [19]:
table = PrettyTable(["Quantidade de camadas","Função de ativação",                  # Criacao da tabela
                      "Hiperparametro", "Quantidade de épocas", 
                      "Quantidade de neurônios", "Disposição dos neurônios"])        

table.padding_width = 1

for i in configuracoes:                                                           # loop para preencher a tabela com os dados
  table.add_row([i["camadasOcultas"], i["funcaoAtivacao"], i["hiperparametro"],
                i["epocas"], i["neuronios"], i["disposicaoNeuronios"]])

print("----------------------------------------------------- CONFIGURAÇÕES DAS REDES NEURAIS -----------------------------------------------------")                   # print da matriz de confusao
print(table)


----------------------------------------------------- CONFIGURAÇÕES DAS REDES NEURAIS -----------------------------------------------------
+-----------------------+--------------------+----------------+----------------------+-------------------------+--------------------------+
| Quantidade de camadas | Função de ativação | Hiperparametro | Quantidade de épocas | Quantidade de neurônios | Disposição dos neurônios |
+-----------------------+--------------------+----------------+----------------------+-------------------------+--------------------------+
|           1           |        tanh        |      sgd       |         150          |            11           |            11            |
|           1           |        tanh        |      adam      |         100          |            8            |            8             |
|           1           |        tanh        |      sgd       |         150          |            13           |            13            |
|           1       

#### Treinamento das redes com os parametros listados acima

In [67]:
melhoresMatrizes = []
desempenhoGeral = []
melhorDesempenhoIndividual = []
cont=0
for i in configuracoes:
    cont+=1
    f1scores = np.zeros(10)
    acuracias = np.zeros(10)

    clf = MLPClassifier(hidden_layer_sizes=(i["disposicaoNeuronios"]),
                            activation=i["funcaoAtivacao"], 
                            solver=i["hiperparametro"], 
                            max_iter=i["epocas"], 
                            verbose=False)

    for j in range(10):
        x_train, x_test, y_train, y_test = train_test_split(
            x_preditor, y_alvo, test_size=0.3, train_size=0.7)
        
        X_train_std = (x_train - np.mean(x_train))/np.std(x_train)
        X_test_std = (x_test - np.mean(x_train))/np.std(x_train)

        clf.fit(X_train_std, y_train)

        y_pred = clf.predict(X_test_std)

        f1scores[j] = f1_score(y_test, y_pred, average='macro')
        acuracias[j] = accuracy_score(y_test, y_pred)

        if j == 0:
          melhorDesempenhoIndividual.append({
              "acuracia": acuracias[j],
              "fscore": f1scores[j]
          })
          melhoresMatrizes.append(confusion_matrix(y_test, y_pred))
        else:
          if acuracias[j] >= melhorDesempenhoIndividual[cont-1]["acuracia"]:
            if  acuracias[j] == melhorDesempenhoIndividual[cont-1]["acuracia"]:
              if f1scores[j] > melhorDesempenhoIndividual[cont-1]["fscore"]:
                melhorDesempenhoIndividual[cont-1]["fscore"] = f1scores[j]
                melhorDesempenhoIndividual[cont-1]["acuracia"] = acuracias[j]
                melhoresMatrizes[cont-1] = confusion_matrix(y_test, y_pred)
            else:
              melhorDesempenhoIndividual[cont-1]["fscore"] = f1scores[j]
              melhorDesempenhoIndividual[cont-1]["acuracia"] = acuracias[j]
              melhoresMatrizes[cont-1] = confusion_matrix(y_test, y_pred)

    desempenhoGeral.append({
        "configuracao": cont,
        "media": round(acuracias.mean(), 5),
        "fscore": round(f1scores.mean(), 5)
    })

In [61]:
# desempenhoGeral
sorted_list = sorted(desempenhoGeral, key=lambda k: k['media']) 
primeiro = sorted_list[9]
segundo = sorted_list[8]
terceiro = sorted_list[7]

table = PrettyTable(["", "Camadas","Função de ativação",                  # Criacao da tabela
                      "Hiperparametro", "Épocas", 
                      "Quantidade de neurônios", "Disposição dos neurônios", 
                     "Média acuracia", "Média F-Score"])        

table.padding_width = 1

# preenchimento dos dados da table (3 melhores desempenhos)
table.add_row(["1°", configuracoes[primeiro["configuracao"]-1]["camadasOcultas"], 
               configuracoes[primeiro["configuracao"]-1]["funcaoAtivacao"], 
               configuracoes[primeiro["configuracao"]-1]["hiperparametro"], 
               configuracoes[primeiro["configuracao"]-1]["epocas"], 
               configuracoes[primeiro["configuracao"]-1]["neuronios"], 
               configuracoes[primeiro["configuracao"]-1]["disposicaoNeuronios"],
               primeiro["media"], primeiro["fscore"]])
table.add_row(["2°", configuracoes[segundo["configuracao"]-1]["camadasOcultas"], 
               configuracoes[segundo["configuracao"]-1]["funcaoAtivacao"], 
               configuracoes[segundo["configuracao"]-1]["hiperparametro"], 
               configuracoes[segundo["configuracao"]-1]["epocas"], 
               configuracoes[segundo["configuracao"]-1]["neuronios"], 
               configuracoes[segundo["configuracao"]-1]["disposicaoNeuronios"],
               segundo["media"], segundo["fscore"]])
table.add_row(["3°", configuracoes[terceiro["configuracao"]-1]["camadasOcultas"], 
               configuracoes[terceiro["configuracao"]-1]["funcaoAtivacao"], 
               configuracoes[terceiro["configuracao"]-1]["hiperparametro"], 
               configuracoes[terceiro["configuracao"]-1]["epocas"], 
               configuracoes[terceiro["configuracao"]-1]["neuronios"], 
               configuracoes[terceiro["configuracao"]-1]["disposicaoNeuronios"],
               terceiro["media"], terceiro["fscore"]])

print("---------------------------------------------------------- CONFIGURAÇÕES DAS REDES NEURAIS ----------------------------------------------------------")                   # print da matriz de confusao
print(table)

# Print das matrizes de confusao das melhores configuracões
print("------------- Primeiro Colocado ------------")
print(melhoresMatrizes[primeiro["configuracao"]-1])

print("\n------------- Segundo Colocado -------------")
print(melhoresMatrizes[segundo["configuracao"]-1])

print("\n------------- Terceiro Colocado ------------")
print(melhoresMatrizes[terceiro["configuracao"]-1])

---------------------------------------------------------- CONFIGURAÇÕES DAS REDES NEURAIS ----------------------------------------------------------
+----+---------+--------------------+----------------+--------+-------------------------+--------------------------+----------------+---------------+
|    | Camadas | Função de ativação | Hiperparametro | Épocas | Quantidade de neurônios | Disposição dos neurônios | Média acuracia | Média F-Score |
+----+---------+--------------------+----------------+--------+-------------------------+--------------------------+----------------+---------------+
| 1° |    1    |      logistic      |      adam      |  200   |            15           |            15            |    0.73191     |    0.54205    |
| 2° |    1    |        tanh        |      sgd       |  150   |            13           |            13            |    0.72023     |    0.47931    |
| 3° |    1    |        tanh        |      sgd       |  150   |            11           |           

## Estimando o número de neurônios

Um dos problemas de pesquisa com redes neurais artificiais consiste na determinação do número de neurônios em sua arquitetura. Embora não seja possível definir a priori qual rede neural é adequada para um problema, pois isto só é possível mediante uma busca exaustiva, há regras na literatura que sugerem o número de neurônios escondidos, tal como a regra da Pirâmide Geométrica, dada a seguir:

$$N_h = \alpha \cdot \sqrt{N_i \cdot N_o},$$

em que $N_h$ é o número de neurônios ocultos (a serem distribuídos em uma ou duas camadas ocultas), $N_i$ é o número de neurônios na camada de entrada e $N_o$ é o número de neurônios na camada de saída. 

1. Consulte a documentação da classe MLPClassifier (disponível em https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) e obtenha os valores de $N_i$ e $N_h$.
2. Teste os valores de $\alpha$ como sendo iguais a $0.5$, $2$ e $3$.
3. Proponha pelo menos 30 redes neurais segundo a regra da pirâmide geométrica e teste-as nos mesmos termos estabelecidos anterioremente  (solver, épocas, etc.)  
    3.1 Apresente as top-3 melhores redes no tocante à F-Score e Acurácia  

#### Como o texto acima indica e segundo a documentação da biblioteca `sklearn.neural_network.MLPClassifier` informa, os valores para **Ni, No** estão relacionados a quantidade de atributos preditores e quantidade de classes respectivamente. Sendo assim ambos correspondem, respectivamente a x e 7.


### Criacao das 30 configurações

In [7]:
# Atribuicao dos parametros da regra da piramide geometrica
Ni = 10
No = 7
alpha = [0.5, 2, 3]
Nh = [] 

for a in alpha:
  Nh.append(int(a*math.sqrt(Ni*No))) 

# print(Nh)
# proposicao das 30 redes neurais usando a regra da pirameide para a quantidade de neuronios
funcaoAtivação = ["identity", "logistic", "tanh", "relu"]                         # declaracao do vetor com os tipos de funcao de ativacao
hiperParametro = ["adam", "sgd"]                                                  # declaracao do vetor com o tipo do hiperparametro solver
epocas = [100, 150, 200]                                                          # declaracao do vetor com a quanitdade de epocas
camadas = [1, 2]
configuracoes = []                                                                # declaracao do vetor que armazenara as configuracoes geradas

for i in range (30):                                                              # loop para gerar aleatoriamente as configuracoes (dentro do especificado)
  tipoEpoca = random.randrange(1, 1000000, 1)%3
  tipoHiperParametro = random.randrange(1, 1000000, 1)%2
  tipoFuncaoAtivacao = random.randrange(1, 1000000, 1)%4
  xqtdCamadas = random.randrange(1, 1000000, 1)%2
  qtdNeuronios = random.randrange(1, 1000000, 1)%3
  
  aux = random.randrange(1, Nh[qtdNeuronios], 1)                                      # variavel auxiliar
  disposicaoNeuronios = Nh[qtdNeuronios] if camadas[xqtdCamadas]==1 else (aux, Nh[qtdNeuronios]-aux) # disposicao dos neuronios nas 2 camadas ocultas

  configuracoes.append({                                                          # armazenamento das configuracoes geradas
    "camadasOcultas": camadas[xqtdCamadas],
    "funcaoAtivacao": funcaoAtivação[tipoFuncaoAtivacao],
    "hiperparametro": hiperParametro[tipoHiperParametro],
    "epocas": epocas[tipoEpoca],
    "neuronios": Nh[qtdNeuronios],
    "disposicaoNeuronios": disposicaoNeuronios
  })

#### Configuracao das 30 redes propostas

In [8]:
table = PrettyTable(["Camadas","Função de ativação",                  # Criacao da tabela
                      "Hiperparametro", "Épocas", 
                      "Quantidade de neurônios", "Disposição dos neurônios"])        

table.padding_width = 1

for i in configuracoes:                                                           # loop para preencher a tabela com os dados
  table.add_row([i["camadasOcultas"], i["funcaoAtivacao"], i["hiperparametro"],
                i["epocas"], i["neuronios"], i["disposicaoNeuronios"]])

print("--------------------------------------- CONFIGURAÇÕES DAS REDES NEURAIS ---------------------------------------")                   # print da matriz de confusao
print(table)


--------------------------------------- CONFIGURAÇÕES DAS REDES NEURAIS ---------------------------------------
+---------+--------------------+----------------+--------+-------------------------+--------------------------+
| Camadas | Função de ativação | Hiperparametro | Épocas | Quantidade de neurônios | Disposição dos neurônios |
+---------+--------------------+----------------+--------+-------------------------+--------------------------+
|    1    |      logistic      |      adam      |  200   |            4            |            4             |
|    2    |        relu        |      adam      |  150   |            25           |         (4, 21)          |
|    2    |      identity      |      sgd       |  200   |            25           |         (20, 5)          |
|    1    |        tanh        |      adam      |  150   |            16           |            16            |
|    1    |        tanh        |      sgd       |  150   |            4            |            4       

#### Criação e treinamento das redes neurais propostas acima

In [9]:
melhoresMatrizes = []
desempenhoGeral = []
melhorDesempenhoIndividual = []
cont=0
for i in configuracoes:
    cont+=1
    f1scores = np.zeros(10)
    acuracias = np.zeros(10)

    clf = MLPClassifier(hidden_layer_sizes=(i["disposicaoNeuronios"]),
                            activation=i["funcaoAtivacao"], 
                            solver=i["hiperparametro"], 
                            max_iter=i["epocas"], 
                            verbose=False)

    for j in range(10):
        x_train, x_test, y_train, y_test = train_test_split(
            x_preditor, y_alvo, test_size=0.3, train_size=0.7)
        
        X_train_std = (x_train - np.mean(x_train))/np.std(x_train)
        X_test_std = (x_test - np.mean(x_train))/np.std(x_train)

        clf.fit(X_train_std, y_train)

        y_pred = clf.predict(X_test_std)

        f1scores[j] = f1_score(y_test, y_pred, average='macro')
        acuracias[j] = accuracy_score(y_test, y_pred)

        if j == 0:
          melhorDesempenhoIndividual.append({
              "acuracia": acuracias[j],
              "fscore": f1scores[j]
          })
          melhoresMatrizes.append(confusion_matrix(y_test, y_pred))
        else:
          if acuracias[j] >= melhorDesempenhoIndividual[cont-1]["acuracia"]:
            if  acuracias[j] == melhorDesempenhoIndividual[cont-1]["acuracia"]:
              if f1scores[j] > melhorDesempenhoIndividual[cont-1]["fscore"]:
                melhorDesempenhoIndividual[cont-1]["fscore"] = f1scores[j]
                melhorDesempenhoIndividual[cont-1]["acuracia"] = acuracias[j]
                melhoresMatrizes[cont-1] = confusion_matrix(y_test, y_pred)
            else:
              melhorDesempenhoIndividual[cont-1]["fscore"] = f1scores[j]
              melhorDesempenhoIndividual[cont-1]["acuracia"] = acuracias[j]
              melhoresMatrizes[cont-1] = confusion_matrix(y_test, y_pred)

    desempenhoGeral.append({
        "configuracao": cont,
        "media": round(acuracias.mean(), 5),
        "fscore": round(f1scores.mean(), 5)
    })

print(desempenhoGeral)
print(melhorDesempenhoIndividual)
print(melhoresMatrizes)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iter

[{'configuracao': 1, 'media': 0.70101, 'fscore': 0.39226}, {'configuracao': 2, 'media': 0.71816, 'fscore': 0.48475}, {'configuracao': 3, 'media': 0.69705, 'fscore': 0.41146}, {'configuracao': 4, 'media': 0.73518, 'fscore': 0.5639}, {'configuracao': 5, 'media': 0.70056, 'fscore': 0.38505}, {'configuracao': 6, 'media': 0.69129, 'fscore': 0.3588}, {'configuracao': 7, 'media': 0.69328, 'fscore': 0.34946}, {'configuracao': 8, 'media': 0.69535, 'fscore': 0.34662}, {'configuracao': 9, 'media': 0.69399, 'fscore': 0.34882}, {'configuracao': 10, 'media': 0.69456, 'fscore': 0.32666}, {'configuracao': 11, 'media': 0.69899, 'fscore': 0.36228}, {'configuracao': 12, 'media': 0.69999, 'fscore': 0.38952}, {'configuracao': 13, 'media': 0.69558, 'fscore': 0.38367}, {'configuracao': 14, 'media': 0.69459, 'fscore': 0.34824}, {'configuracao': 15, 'media': 0.70042, 'fscore': 0.39709}, {'configuracao': 16, 'media': 0.69998, 'fscore': 0.39911}, {'configuracao': 17, 'media': 0.71826, 'fscore': 0.43212}, {'confi

In [20]:
# desempenhoGeral
sorted_list = sorted(desempenhoGeral, key=lambda k: k['media']) 
primeiro = sorted_list[29]
segundo = sorted_list[28]
terceiro = sorted_list[27]

table = PrettyTable(["", "Camadas","Função de ativação",                  # Criacao da tabela
                      "Hiperparametro", "Épocas", 
                      "Quantidade de neurônios", "Disposição dos neurônios", 
                     "Média acuracia", "Média F-Score"])        

table.padding_width = 1

# preenchimento dos dados da table (3 melhores desempenhos)
table.add_row(["1°", configuracoes[primeiro["configuracao"]-1]["camadasOcultas"], 
               configuracoes[primeiro["configuracao"]-1]["funcaoAtivacao"], 
               configuracoes[primeiro["configuracao"]-1]["hiperparametro"], 
               configuracoes[primeiro["configuracao"]-1]["epocas"], 
               configuracoes[primeiro["configuracao"]-1]["neuronios"], 
               configuracoes[primeiro["configuracao"]-1]["disposicaoNeuronios"],
               primeiro["media"], primeiro["fscore"]])
table.add_row(["2°", configuracoes[segundo["configuracao"]-1]["camadasOcultas"], 
               configuracoes[segundo["configuracao"]-1]["funcaoAtivacao"], 
               configuracoes[segundo["configuracao"]-1]["hiperparametro"], 
               configuracoes[segundo["configuracao"]-1]["epocas"], 
               configuracoes[segundo["configuracao"]-1]["neuronios"], 
               configuracoes[segundo["configuracao"]-1]["disposicaoNeuronios"],
               segundo["media"], segundo["fscore"]])
table.add_row(["3°", configuracoes[terceiro["configuracao"]-1]["camadasOcultas"], 
               configuracoes[terceiro["configuracao"]-1]["funcaoAtivacao"], 
               configuracoes[terceiro["configuracao"]-1]["hiperparametro"], 
               configuracoes[terceiro["configuracao"]-1]["epocas"], 
               configuracoes[terceiro["configuracao"]-1]["neuronios"], 
               configuracoes[terceiro["configuracao"]-1]["disposicaoNeuronios"],
               terceiro["media"], terceiro["fscore"]])

print("---------------------------------------------------------- CONFIGURAÇÕES DAS REDES NEURAIS ----------------------------------------------------------")                   # print da matriz de confusao
print(table)

# Print das matrizes de confusao das melhores configuracões
print("------------- Primeiro Colocado ------------")
print(melhoresMatrizes[primeiro["configuracao"]-1])

print("\n------------- Segundo Colocado -------------")
print(melhoresMatrizes[segundo["configuracao"]-1])

print("\n------------- Terceiro Colocado ------------")
print(melhoresMatrizes[terceiro["configuracao"]-1])

---------------------------------------------------------- CONFIGURAÇÕES DAS REDES NEURAIS ----------------------------------------------------------
+----+---------+--------------------+----------------+--------+-------------------------+--------------------------+----------------+---------------+
|    | Camadas | Função de ativação | Hiperparametro | Épocas | Quantidade de neurônios | Disposição dos neurônios | Média acuracia | Média F-Score |
+----+---------+--------------------+----------------+--------+-------------------------+--------------------------+----------------+---------------+
| 1° |    1    |        tanh        |      adam      |  150   |            25           |            25            |    0.74959     |    0.60096    |
| 2° |    1    |      logistic      |      adam      |  100   |            25           |            25            |    0.74573     |    0.58238    |
| 3° |    2    |      logistic      |      adam      |  200   |            25           |         (2

## Testando as Redes Neurais com Atributos Categóricos

1. Considere as 6 redes neurais obtidas nos dois top-3 anteriores (arquiteturas próprias e regra da pirâmide geométrica)
2. Com todos os atributos preditores da base de dados original, incluindo os categóricos, treine e teste estas mesmas redes por 100 repetições  
    2.1 Considere o melhor otimizador para cada uma delas  
    2.2 Faça uso de 200 épocas para treinamento  
    2.2 Apresente os resultados de acurácia e F-Score em termos da média +- dp para cada arquitetura
3. Apresente o gráfico boxplot para o F-Score das 6 arquiteturas perante as 100 repetições

In [22]:
# criacao do vetor com as  melhores configuracoes da arquitetura propria
top3_arquitetura = []
top3_piramide = []

# preenchimento do vetor com as 3 melhores configuracoes da arquitetura propria
top3_arquitetura.append({                                                          
    "camadasOcultas": 1,"funcaoAtivacao": "logistic", "hiperparametro": "adam",
    "epocas": 200, "neuronios": 15, "disposicaoNeuronios": 15
  })
top3_arquitetura.append({                                                          
    "camadasOcultas": 1, "funcaoAtivacao": "tanh", "hiperparametro": "sgd",
    "epocas": 200, "neuronios": 13, "disposicaoNeuronios": 13
  })
top3_arquitetura.append({                                                          
    "camadasOcultas": 1, "funcaoAtivacao": "tanh", "hiperparametro": "sgd",
    "epocas": 200, "neuronios": 11, "disposicaoNeuronios": 11
  })

# preenchimento do vetor com as 3 melhores configuracoes da arquitetura priramide
top3_piramide.append({                                                          
    "camadasOcultas": 1,"funcaoAtivacao": "tanh", "hiperparametro": "adam",
    "epocas": 200, "neuronios": 25, "disposicaoNeuronios": 25
  })
top3_piramide.append({                                                          
    "camadasOcultas": 1, "funcaoAtivacao": "logistic", "hiperparametro": "adam",
    "epocas": 200, "neuronios": 25, "disposicaoNeuronios": 25
  })
top3_piramide.append({                                                          
    "camadasOcultas": 2, "funcaoAtivacao": "logistic", "hiperparametro": "adam",
    "epocas": 200, "neuronios": 25, "disposicaoNeuronios": (22, 3)
  })

In [24]:
# configuracoes_top6 = [] # Vetor de configuracoes do top 6
top_6 = []              # Vetor dos top 6
top_6 = top3_arquitetura + top3_piramide

ranking_top6 = []
fscore_top6 = []
acuracia_top6 = []
cont=0
for i in top_6:
    fscore_top6.append([])
    acuracia_top6.append([])
    cont+=1
    f1scores = np.zeros(10)
    acuracias = np.zeros(10)

    clf = MLPClassifier(hidden_layer_sizes=(i["disposicaoNeuronios"]),
                            activation=i["funcaoAtivacao"], 
                            solver=i["hiperparametro"], 
                            max_iter=200, #sao 200 epocas 
                            verbose=False)
    for j in range(10):
        x_train, x_test, y_train, y_test = train_test_split(
            x_preditor, y_alvo, test_size=0.3, train_size=0.7)
        
        X_train_std = (x_train - np.mean(x_train))/np.std(x_train)
        X_test_std = (x_test - np.mean(x_train))/np.std(x_train)

        clf.fit(X_train_std, y_train)

        y_pred = clf.predict(X_test_std)

        fscore_top6[cont - 1].append(f1_score(y_test, y_pred, average='macro')) ## Calcula o fscore e coloca no score do top6
        acuracia_top6[cont - 1].append(accuracy_score(y_test, y_pred)) ## Calcula a acuracia e coloca no score do top6

    ranking_top6.append({
        "configuracao": cont,
        "mediaAcuracia": acuracia_top6[cont-1].mean(),
        "desvioacuracia": acuracia_top6[cont-1].std()
        "mediafscore": fscore_top6[cont-1].mean(),
        "desvioFscore": fscore_top6[cont-1].std()
    })

In [ ]:
# boxplot_top6 = pd.DataFrame(fscore_top6) #pegar o fscore e botar num dataframe
# boxplot_top6 = boxplot_top6.transpose() #arrumar o dataframe
 
# boxplot_top6.boxplot() # plotar em um boxplot

## Considerações Parciais

1. É possível identificar uma rede com desempenho superior às demais?
2. Qual estratégia mostrou-se mais producente para a obtenção de boas arquiteturas (Estratégia Própria ou Pirâmide Geométrica)? Por quê?
3. Considerar os atributos categóricos trouxe melhorias? Justifique.
4. Um número maior de épocas trouxe melhorias?
5. Qual a maior dificuldade de resolução do problema proposto perante as RNAs?